In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

from get_db_url import get_db_url

# Zillow Clustering Exercises

---

## Acquire

### 1

Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
sql = '''
SELECT
    properties_2017.*,
    logerror,
    transactiondate,
    typeconstructiondesc,
    airconditioningdesc,
    architecturalstyledesc,
    buildingclassdesc,
    propertylandusedesc,
    storydesc,
    heatingorsystemdesc
FROM properties_2017
JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
    AND predictions_2017.transactiondate LIKE '2017%%'
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN storytype USING (storytypeid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
    
JOIN (
    SELECT
        parcelid,
        MAX(transactiondate) AS date
    FROM predictions_2017
    GROUP BY parcelid
) AS max_dates ON properties_2017.parcelid = max_dates.parcelid
    AND predictions_2017.transactiondate = max_dates.date
    
WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
'''

zillow = pd.read_sql(sql, get_db_url('zillow'))

In [3]:
# zillow = zillow.drop(columns = ['parcelid', 'id'])

### 2

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
zillow.shape

(77380, 68)

In [5]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [6]:
zillow.describe().T

,count,mean,std,min,25%,50%,75%,max
id,77380.0,1.495126e+06,8.609057e+05,3.490000e+02,7.520500e+05,1.497870e+06,2.240480e+06,2.982274e+06
parcelid,77380.0,1.300715e+07,3.481368e+06,1.071186e+07,1.153830e+07,1.253155e+07,1.421184e+07,1.676893e+08
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03
bathroomcnt,77380.0,2.299134e+00,9.966566e-01,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
bedroomcnt,77380.0,3.053489e+00,1.139103e+00,0.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.600000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
buildingqualitytypeid,49671.0,6.534638e+00,1.721933e+00,1.000000e+00,6.000000e+00,6.000000e+00,8.000000e+00,1.200000e+01
calculatedbathnbr,76771.0,2.316871e+00,9.797606e-01,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01


In [7]:
dict(zillow.dtypes)

{'id': dtype('int64'),
 'parcelid': dtype('int64'),
 'airconditioningtypeid': dtype('float64'),
 'architecturalstyletypeid': dtype('float64'),
 'basementsqft': dtype('float64'),
 'bathroomcnt': dtype('float64'),
 'bedroomcnt': dtype('float64'),
 'buildingclasstypeid': dtype('float64'),
 'buildingqualitytypeid': dtype('float64'),
 'calculatedbathnbr': dtype('float64'),
 'decktypeid': dtype('float64'),
 'finishedfloor1squarefeet': dtype('float64'),
 'calculatedfinishedsquarefeet': dtype('float64'),
 'finishedsquarefeet12': dtype('float64'),
 'finishedsquarefeet13': dtype('float64'),
 'finishedsquarefeet15': dtype('float64'),
 'finishedsquarefeet50': dtype('float64'),
 'finishedsquarefeet6': dtype('float64'),
 'fips': dtype('float64'),
 'fireplacecnt': dtype('float64'),
 'fullbathcnt': dtype('float64'),
 'garagecarcnt': dtype('float64'),
 'garagetotalsqft': dtype('float64'),
 'hashottuborspa': dtype('float64'),
 'heatingorsystemtypeid': dtype('float64'),
 'latitude': dtype('float64'),
 'l

In [8]:
zillow.value_counts()

Series([], dtype: int64)

### 3

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [9]:
def summarize_column_nulls(df):
    return pd.concat([
        zillow.isnull().sum().rename('count'),
        zillow.isnull().mean().rename('percent')
    ], axis = 1)

In [10]:
summarize_column_nulls(zillow)

,count,percent
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52427,0.677526
architecturalstyletypeid,77174,0.997338
basementsqft,77330,0.999354
...,...,...
architecturalstyledesc,77174,0.997338
buildingclassdesc,77365,0.999806
propertylandusedesc,0,0.000000
storydesc,77330,0.999354


### 4

Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [11]:
def summarize_row_nulls(df):
    return pd.concat([
        zillow.isnull().sum(axis = 1).rename('columns_missing'),
        zillow.isnull().mean(axis = 1).rename('percent_missing')
    ], axis = 1).value_counts().sort_index()

In [12]:
summarize_row_nulls(zillow)

columns_missing  percent_missing
23               0.338235               2
24               0.352941              13
25               0.367647              24
26               0.382353              65
27               0.397059             316
28               0.411765             455
29               0.426471            5270
30               0.441176            3455
31               0.455882            9891
32               0.470588           12579
33               0.485294           14782
34               0.500000           13326
35               0.514706            5148
36               0.529412            5775
37               0.544118            3620
38               0.558824            1925
39               0.573529             285
40               0.588235             230
41               0.602941              29
42               0.617647              23
43               0.632353              28
44               0.647059              78
45               0.661765              50
4

## Prepare

In [13]:
# First things first. I'll drop all the id columns.

columns = [
    'typeconstructiontypeid',
    'airconditioningtypeid',
    'architecturalstyletypeid',
    'buildingclasstypeid',
    'propertylandusetypeid',
    'storytypeid',
    'heatingorsystemtypeid'
]
zillow = zillow.drop(columns = columns)

### 1

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [14]:
# Do any rows not have property type column
zillow.propertylandusedesc.isnull().sum()

0

In [15]:
# We'll keep any property type that would be sold as a single unit.
# We'll keep Single Family Residential, Condominium, Cluster Home, Mobile Home, Manufactured Homes, 
# Residential General, and Townhouse.
zillow.propertylandusedesc.value_counts()

Single Family Residential                     52319
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [16]:
property_types = [
    'Single Family Residential',
    'Condominium',
    'Cluster Home',
    'Mobile Home',
    'Manufactured, Modular, Prefabricated Homes',
    'Townhouse'
]
zillow = zillow[zillow.propertylandusedesc.isin(property_types)]

In [17]:
# We'll also remove properties with 0 bathrooms since this seems unusual.
# Is it a communal bathroom? Let's not worry about it.
zillow.bathroomcnt.value_counts()

2.0     29874
3.0     15356
1.0     12790
2.5      6602
4.0      2490
1.5      1415
3.5      1033
5.0       824
4.5       694
6.0       324
0.0       228
5.5       224
7.0        88
8.0        53
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         3
18.0        1
13.0        1
Name: bathroomcnt, dtype: int64

In [18]:
# Finally let's grab properties where unitcnt is 1 or null
zillow = zillow[(zillow.unitcnt == 1) | (zillow.unitcnt.isnull())]

In [19]:
zillow.shape

(72048, 61)

In [20]:
# Let's put it in a function

def get_single_unit_properties(df):
    property_types = [
        'Single Family Residential',
        'Condominium',
        'Cluster Home',
        'Mobile Home',
        'Manufactured, Modular, Prefabricated Homes',
        'Townhouse'
    ]
    df = df[df.propertylandusedesc.isin(property_types)]
    
    df = df[(df.unitcnt == 1) | (df.unitcnt.isnull())]
    
    return df

### 2

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.

In [21]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    df = df.dropna(axis = 'columns', thresh = round(df.shape[0] * prop_required_column))
    df = df.dropna(axis = 'index', thresh = round(df.shape[1] * prop_required_row))
    
    return df

In [22]:
zillow = handle_missing_values(zillow, 0.90, 0.90)

### 3

Decide how to handle the remaining missing values:

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

In [23]:
# Let's see what nulls we have left
zillow.isnull().sum()

id                                 0
parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                 80
calculatedfinishedsquarefeet       9
finishedsquarefeet12             216
fips                               0
fullbathcnt                       80
latitude                           0
longitude                          0
propertycountylandusecode          0
rawcensustractandblock             0
regionidcity                    1317
regionidcounty                     0
regionidzip                       34
roomcnt                            0
yearbuilt                         43
structuretaxvaluedollarcnt        70
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          5
censustractandblock              201
logerror                           0
transactiondate                    0
propertylandusedesc                0
d

In [24]:
from sklearn.impute import SimpleImputer
from preprocessing import split_data

In [25]:
# calculatedbathnbr, impute with mode
# calculatedfinishedsquarefeet, impute with mean
# finishedsquarefeet12, impute with mean
# fullbathcnt, impute with mode
# regionidcity, impute with mode
# regionidzip, impute with mode
# yearbuilt, impute with mode
# structuretaxvaluedollarcnt, impute with mean
# taxvaluedollarcnt, impute with mean
# landtaxvaluedollarcnt, impute with mean
# taxamount, impute with mean
# censustractandblock, impute with median

In [26]:
train, validate, test = split_data(zillow)

In [27]:
train.isnull().sum()

id                                0
parcelid                          0
bathroomcnt                       0
bedroomcnt                        0
calculatedbathnbr                49
calculatedfinishedsquarefeet      5
finishedsquarefeet12            128
fips                              0
fullbathcnt                      49
latitude                          0
longitude                         0
propertycountylandusecode         0
rawcensustractandblock            0
regionidcity                    737
regionidcounty                    0
regionidzip                      23
roomcnt                           0
yearbuilt                        25
structuretaxvaluedollarcnt       50
taxvaluedollarcnt                 1
assessmentyear                    0
landtaxvaluedollarcnt             1
taxamount                         3
censustractandblock             127
logerror                          0
transactiondate                   0
propertylandusedesc               0
dtype: int64

In [28]:
columns = [
    'calculatedfinishedsquarefeet',
    'finishedsquarefeet12',
    'structuretaxvaluedollarcnt',
    'taxvaluedollarcnt',
    'landtaxvaluedollarcnt',
    'taxamount'
]

imputer = SimpleImputer(strategy = 'mean')
imputer.fit(train[columns])

train[columns] = imputer.transform(train[columns])
validate[columns] = imputer.transform(validate[columns])
test[columns] = imputer.transform(test[columns])

/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [29]:
columns = [
    'calculatedbathnbr',
    'fullbathcnt',
    'regionidcity',
    'regionidzip',
    'yearbuilt'
]

imputer = SimpleImputer(strategy = 'most_frequent')
imputer.fit(train[columns])

train[columns] = imputer.transform(train[columns])
validate[columns] = imputer.transform(validate[columns])
test[columns] = imputer.transform(test[columns])

/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [30]:
columns = [
    'censustractandblock'
]

imputer = SimpleImputer(strategy = 'median')
imputer.fit(train[columns])

train[columns] = imputer.transform(train[columns])
validate[columns] = imputer.transform(validate[columns])
test[columns] = imputer.transform(test[columns])

/usr/local/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [31]:
def impute_missing_values(df, columns_strategy):
    train, validate, test = split_data(df)
    
    for strategy, columns in columns_strategy.items():
        imputer = SimpleImputer(strategy = strategy)
        imputer.fit(train[columns])

        train[columns] = imputer.transform(train[columns])
        validate[columns] = imputer.transform(validate[columns])
        test[columns] = imputer.transform(test[columns])
        
    return train, validate, test

In [32]:
# columns_strategy = {
#     'mean' : [
#         'calculatedfinishedsquarefeet',
#         'finishedsquarefeet12',
#         'structuretaxvaluedollarcnt',
#         'taxvaluedollarcnt',
#         'landtaxvaluedollarcnt',
#         'taxamount'
#     ],
#     'most_frequent' : [
#         'calculatedbathnbr',
#         'fullbathcnt',
#         'regionidcity',
#         'regionidzip',
#         'yearbuilt'
#     ],
#     'median' : [
#         'censustractandblock'
#     ]
# }

# train, validate, test = impute_missing_values(zillow, columns_strategy)

In [33]:
train.isnull().sum()

id                              0
parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
propertylandusedesc             0
dtype: int64

# Mall Customers

In [34]:
from wrangle_mall import *

## 1

Acquire data from mall_customers.customers in mysql database.

In [35]:
customers = get_mall_data()
customers.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


## 2

Summarize data (include distributions and descriptive statistics).

In [36]:
summarize_data(customers)

Shape: (200, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB

 None

        customer_id         age  annual_income  spending_score
count   200.000000  200.000000     200.000000      200.000000
mean    100.500000   38.850000      60.560000       50.200000
std      57.879185   13.969007      26.264721       25.823522
min       1.000000   18.000000      15.000000        1.000000
25%      50.750000   28.750000      41.500000       34.750000
50%     100.500000   36.000000      61.500000       50.000000
75%     150.250000   49.000000      78.000000       73.000000
max     200.00000

## 3

Detect outliers using IQR.

In [37]:
get_upper_outliers(customers.annual_income, 1.5)

0      0.00
1      0.00
2      0.00
3      0.00
4      0.00
       ... 
195    0.00
196    0.00
197    0.00
198    4.25
199    4.25
Name: annual_income, Length: 200, dtype: float64

In [38]:
get_lower_outliers(customers.annual_income, 1.5)

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    0
199    0
Name: annual_income, Length: 200, dtype: int64

## 4

Split data (train, validate, and test split).

In [39]:
train, validate, test = split_data(customers)

## 5

Encode categorical columns using a one hot encoder (pd.get_dummies).

In [40]:
train = encode_gender(train)
validate = encode_gender(validate)
test = encode_gender(test)

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 131 to 70
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     112 non-null    int64  
 1   age             112 non-null    float64
 2   annual_income   112 non-null    float64
 3   spending_score  112 non-null    float64
 4   gender_Male     112 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 5.2 KB


## 6

Handles missing values.

In [41]:
train.isnull().sum()

customer_id       0
age               0
annual_income     0
spending_score    0
gender_Male       0
dtype: int64

Handled!

## 7

Scaling

In [42]:
from preprocessing import scale_data

In [43]:
train, validate, test = scale_data(train, validate, test, train.drop(columns = 'customer_id').columns)

In [44]:
train.head()

,customer_id,age,annual_income,spending_score,gender_Male
131,132,0.403846,0.459016,0.770833,1.0
119,120,0.615385,0.426230,0.583333,0.0
127,128,0.423077,0.459016,0.979167,1.0
93,94,0.423077,0.368852,0.406250,0.0
104,105,0.596154,0.385246,0.572917,1.0
